# ONT sequencing data processing for prokaryote isolates

# Assembly


***

## Assembly

There a several options for assemblers for long read data. In this example, we will use *Flye*

Useful outputs:

- `assembly.fasta`: draft assembly
- `tail flye.log`: basic assembly stats
- `assembly_info.txt`: more info on each contig in the assembly

NOTE:

- *Flye* has an option to account for uneven coverage (e.g. metagenomics data or data generated from non-pure isolates): `--meta`. This can optionally be included in case there are more than one organism in what might be presumed to be a pure isolate (or if the originating sample was already known to be mixed (i.e. more than one genome in the sample))
- *Flye* has three *nano* options
  - `nano_corr`: Assumes data are pre-cleaned via, e.g., illumina reads, prior to assembly
  - `nano_raw`: Assumes older poor quality reads, but can also use on HAC data
  - `nano_hq`: Good for SUP-called reads, but can also be used for HAC reads
    - When working with HAC data, this option *might* return more split contigs. However, preliminary testing in our group did not suggested an appreciable difference between `nano_raw` and `nano_hq` with HAC data. You may wish to try both with your data and assess the assemblies generated.


#### Run *Flye* assemblies

Run *Flye* assemblies for each barcode. These can be relatively quick, so could be run in a loop, but this can also be sped up by running via slurm array (`#SBATCH --array=1-96`) as in the example below.

NOTE:

- Running *Flye* with `--meta` flag and `--nano-hq` setting

In [ ]:
#!/bin/bash
#SBATCH -A <your_project_account>
#SBATCH -J 2_assembly
#SBATCH --time 00:45:00
#SBATCH --array=1-96
#SBATCH --ntasks=1
#SBATCH --mem 10GB
#SBATCH --cpus-per-task=16
#SBATCH -e 2_assembly_%a.err
#SBATCH -o 2_assembly_%a.out

cd /working/dir
mkdir -p 2.assembly/1.assembly.flye

module purge
module load Flye/2.9-gimkl-2020a-Python-3.8.2

srun flye --meta -t ${SLURM_CPUS_PER_TASK} \
--nano-hq 1.trimmed_and_filtered_reads/1.chopper/fastq_files/S${SLURM_ARRAY_TASK_ID}.chopper.fastq \
--o 2.assembly/1.assembly.flye/S${SLURM_ARRAY_TASK_ID}.assembly


Note: Some assemblies may fail (e.g. where no contigs are assembled). This may be due to, for example; poor coverage for this isolate; insufficient data (similar to poor coverage); mixed genomes in a single sample that the assembler is struggling to resolve. Take a note of failed isolateIDs, as this will be useful to know to exclude these isolates downstream.


#### Generate summary table of assembly_info.txt outputs

Merge *Flye* `assembly_info.txt` outputs into one summary table 

Example python code for compiling all assembly_info.txt files into a summary table. 

Note:

- The code below includes a try-execpt clause to allow for cases where assembly failed for some isolates.
- Modify `range(1,96)` to match your sampleIDs

In [ ]:
cd /working/dir

module purge
module load Python/3.8.2-gimkl-2020a
python3

import pandas as pd
import numpy as np

# Compile flye assembly_info results
results_list = []
for i in range(1,96):
    try:
        tmp_df = pd.read_csv('2.assembly/1.assembly.flye/S'+str(i)+'.assembly/assembly_info.txt', sep='\t').rename({'#seq_name': 'contigID'}, axis=1)    
    except FileNotFoundError:
        continue
    tmp_df.index = ['S'+str(i)]*len(tmp_df)
    tmp_df = tmp_df.rename_axis('sampleID').reset_index()
    results_list.append(tmp_df)

# Generate summary table and write out
results_df = pd.concat(results_list, axis=0)
# counts of contigs per sample
counts_df = results_df.groupby('sampleID').size().reset_index(name='perSample_contig_counts')
results_df = pd.merge(results_df, counts_df, on = 'sampleID')
results_df.to_csv('2.assembly/1.assembly.flye/assembly_info.sumary_table.tsv', sep='\t', index=False)

quit()


***

## Long-read polishing


Long-read polishing takes the assemblies that were generated above and maps reads to them and checks and revises them. There are several options for this step, such as *racon* and *medaka*. In the example below, we will use *medaka*.

*medaka* is Oxford Nanopore's in-house assembly polisher. It takes sequences, assembly files, and the model used for basecalling, and generates polished assembly files.

NOTE:

- to find model options, run: `medaka tools list_models`, and then select the applicable model for your basecalling on this specific run.
- For this example data, models and software versions used for basecalling:
  - Basecalling software: `Dorado/0.7.3`
  - Basecalling model: `dna_r10.4.1_e8.2_400bps_sup@v5.0.0` 
  - Kit: `--kit-name SQK-RBK114-96`
  - for latest medaka in NeSI at the time (1.11.1), the latest model available = `r1041_e82_400bps_sup_v4.2.0`

#### Run *medaka* long read polishing of *Flye* assembly

Run *medaka* polishing and then copy all polished assembly files into one output directory

NOTE:

- In this example, the basecalling model was `dna_r10.4.1_e8.2_400bps_sup@v5.0.0`. However, the latest model available in *medaka* at the time was `r1041_e82_400bps_sup_v4.2.0`. Update `-m r1041_e82_400bps_sup_v4.2.0` if the basecalling model was different for your run.
- If some samples failed the assembly step, you could exclude these from the slurm array in this script, as they will only result in failed surm jobs here.


In [ ]:
#!/bin/bash
#SBATCH -A <>
#SBATCH -J 1_assembly_polish
#SBATCH --time 06:00:00
#SBATCH --ntasks=1
#SBATCH --mem 30GB
#SBATCH --array=1-96
#SBATCH --cpus-per-task=8
#SBATCH -e 1_assembly_polish_%a.err
#SBATCH -o 1_assembly_polish_%a.out

# Working directory
cd /work/dir
mkdir -p 2.assembly/2.assembly.LR_polished

# load module
module purge
module load medaka/1.11.1-Miniconda3-22.11.1-1

# Run medaka
medaka_consensus -f -t ${SLURM_CPUS_PER_TASK} \
-m r1041_e82_400bps_sup_v4.2.0 \
-i 0.raw_data/2.basecalled.demux/S${SLURM_ARRAY_TASK_ID}.fastq \
-d 2.assembly/1.assembly.flye/S${SLURM_ARRAY_TASK_ID}.assembly/assembly.fasta \
-o 2.assembly/2.assembly.LR_polished/S${SLURM_ARRAY_TASK_ID}.assembly.polished


#### Copy polished assembly files into one directory and rename fna files with sampleID

- n.b. can ignore `No such file or directory` error messages (assuming they match the samples that failed to assemble any contigs in the assembly step above)

In [ ]:
cd /work/dir
mkdir -p 2.assembly/2.assembly.LR_polished.fna_files

for i in {1..96}; do
    cp 2.assembly/2.assembly.LR_polished/S${i}.assembly.polished/consensus.fasta \
    2.assembly/2.assembly.LR_polished.fna_files/S${i}.assembly.polished.fna
done


#### Evaluating polished assemblies

It always pays to assess the quality of final assemblies. There as numerous tools to do this and metrics you can assess. Here we will get basic stats via contig counts, contig lengths, and NL50 stats via `stats.sh` from the *BBMap* suite of tools. We can then use a bash script to generate summary table from BBMap's stats.sh output.

NOTE: 

- The script below runs in a loop over all 96 sampleIDs, but some samples may not have files available here if they failed to assemble any contigs in the assembly step. We can ignore the following error message: `Exception in thread "main" java.lang.RuntimeException: Input file does not appear to be valid` (assuming they match the samples that failed to assemble any contigs in the assembly step above)


In [ ]:
# Working directory
cd /work/dir
mkdir -p 2.assembly/2.assembly.LR_polished.assembly_stats

# load module
module purge
module load BBMap/38.95-gimkl-2020a

for i in {1..96}; do
    stats.sh in=2.assembly/2.assembly.LR_polished.fna_files/S${i}.assembly.polished.fna > 2.assembly/2.assembly.LR_polished.assembly_stats/S${i}.assembly.polished.stats
done

# Extract key stats from outputs and compile into single summary table
echo -e "sampleID\tcontig_count\tcontig_length_max\tcontig_length_total\tcontig_NL50" \
> 2.assembly/2.assembly.LR_polished.assembly_stats/summary_table.polished_assembly.stats.tsv
for i in {1..96}; do
    infile="2.assembly/2.assembly.LR_polished.assembly_stats/S${i}.assembly.polished.stats"
    SampleID="S${i}"
    contig_count=$(sed -n -e 's/Main genome contig total:[[:space:]]\+//p' ${infile})
    contig_length_max=$(sed -n -e 's/Max contig length:[[:space:]]\+//p' ${infile})
    contig_length_total=$(sed -n -e 's/Main genome contig sequence total:[[:space:]]\+\(.*MB\).*/\1/p' ${infile})
    contig_NL50=$(sed -n -e 's/Main genome contig N\/L50:[[:space:]]\+//p' ${infile})
    echo -e "${SampleID}\t${contig_count}\t${contig_length_max}\t${contig_length_total}\t${contig_NL50}" \
    >> 2.assembly/2.assembly.LR_polished.assembly_stats/summary_table.polished_assembly.stats.tsv
done


***

## Short-read polishing

Historically, Nanopore long-read sequencing has suffered from high error rates, and methods were developed to polish long-read assemblies via adding high quality short read (such as Illumina HiSeq) data as well. Some assembly algorithms have also been developed that take *both* long read and short read data together when generating assemblies (i.e. hybrid assemblies). 

Recent advances have considerably increased Nanopore basecalling quality, and ensuring a high sequencing coverage across the genome also likely helps mitigate some of the remaining errors. 

In one small pilot test within our group, *checkM* completeness and contamination stats were comparable between only long-read polished isolate data and both long- and short-read polished data. Although, the quality and consistency at the individual base level was not tested, and how important this is for your study may depend on what you're asking of the data downstream. 

We will not cover the process of short-read polishing in these docs, but it is worth considering for your data and study question whether you wish to include this step here. Ultimately, if you have high quality short-read data available, or it is easily obtainable, it is likely to be beneficial to incorporate them here. 

***

## Assembly post-processing



#### Add isolateIDs to contig headers

It can be useful for downstream applications (particularly where assembled contigs from all sequenced isolates are analysed together) to add isolateIDs into the headers of contigIDs for each isolate assembly.

The script below uses *sed* to do this in place on the `2.assembly.LR_polished.fna_files`. (If you wish to keep these original files unedited, you can remove the `-i` flag and instead read the result into a new file via `>`)

In [ ]:
cd /working/dir

for i in {1..96}; do
    sed -i -e "s/>/>S${i}_/g" 2.assembly/2.assembly.LR_polished.fna_files/S${i}.assembly.polished.fna
done


#### Filtering out short contigs from assemblies

Short contigs often contain little biologically useful information and can create additional noise in downstream analyses. In this example we will filter these out via *seqmagick*.


In [ ]:
cd /work/dir

module purge
module load seqmagick/0.8.4-gimkl-2020a-Python-3.8.2

mkdir -p 2.assembly/2.assembly.LR_polished.fna_files.m1000

for file in 2.assembly/2.assembly.LR_polished.fna_files/*.fna; do
    filename=$(basename ${file} .fna)
    seqmagick convert --min-length 1000 ${file} 2.assembly/2.assembly.LR_polished.fna_files.m1000/${filename}.m1000.fna
done



#### Generate assembly2contig_lookupTable

A lookup table mapping all contig IDs to assembly IDs for all assemblies can be useful in some cases for downstream applications.

If this is of use, you can use the python code below to generate `assembly2contig_lookupTable.txt`

In [ ]:
# Working directory
cd /work/dir

# Load python
module purge
module load Python/3.8.2-gimkl-2020a
python3

### Import required libraries
import pandas as pd
import numpy as np
from Bio.SeqIO.FastaIO import SimpleFastaParser
import re
import os

# Compile genome2contig lookup tables
with open('2.assembly/2.assembly.LR_polished/assembly2contig_lookupTable.tsv', 'w') as write_file:
    # header
    write_file.write('assemblyID\tcontigID\n')
    # Prok bins
    assemblyfiles_directory = os.fsencode('2.assembly/2.assembly.LR_polished.fna_files')
    for file in os.listdir(assemblyfiles_directory):
        filename = os.fsdecode(file)
        assemblyID = os.path.splitext(filename)[0]
        with open('2.assembly/2.assembly.LR_polished.fna_files/' + filename, 'r') as read_fasta:
            for name, seq in SimpleFastaParser(read_fasta):
                write_file.write(assemblyID + '\t' + name + '\n')

# Compile genome2contig lookup tables - m1000 filtered contigs
with open('2.assembly/2.assembly.LR_polished/assembly2contig_lookupTable.m1000.tsv', 'w') as write_file:
    # header
    write_file.write('assemblyID\tcontigID\n')
    # Prok bins
    assemblyfiles_directory = os.fsencode('2.assembly/2.assembly.LR_polished.fna_files.m1000')
    for file in os.listdir(assemblyfiles_directory):
        filename = os.fsdecode(file)
        assemblyID = os.path.splitext(filename)[0]
        with open('2.assembly/2.assembly.LR_polished.fna_files.m1000/' + filename, 'r') as read_fasta:
            for name, seq in SimpleFastaParser(read_fasta):
                write_file.write(assemblyID + '\t' + name + '\n')

quit()


***